<a href="https://colab.research.google.com/github/mikosa01/TEAMNOLT/blob/main/architeture_updates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import numpy as np 
from pathlib import Path
import sys 
import tensorflow as tf
import pandas as pd 
import matplotlib.pyplot  as plt 
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
downloaded = drive.CreateFile({'id':'1c06v0ViaOvssxGV14zbggB72NcvFR6AE'}) # replace the id with id of file you want to access
downloaded.GetContentFile('Group_Project_Data.zip') 

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
zip_files = ['Train', 'Valid']
for zip_file in zip_files:
  with zipfile.ZipFile('Group_Project_Data.zip'.format(zip_file), 'r') as z: 
    z.extractall('.')
    print('{} unzipped'.format(zip_file))

Train unzipped
Valid unzipped


In [ ]:
train_dir =  '/content/Group_Project_Data/Train'
val_dir =  '/content/Group_Project_Data/Valid'

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255) # rescale pixel values to [0, 1]
val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # resize images to 224x224 pixels
    batch_size=32,
    class_mode='categorical'  # for multi-class classification
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 6000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


1st Architeture

In [ ]:
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
#     tf.keras.layers.MaxPooling2D((2, 2)),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dense(2, activation='softmax')
# ])

In [ ]:
# model.compile(optimizer='adam',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

In [ ]:
# history = model.fit(train_generator, epochs=10, validation_data=val_generator)

Epoch 1/10
188/188 [==============================] - 26s 70ms/step - loss: 0.2057 - accuracy: 0.9475 - val_loss: 0.0056 - val_accuracy: 0.9990
Epoch 2/10
188/188 [==============================] - 14s 75ms/step - loss: 0.0068 - accuracy: 0.9988 - val_loss: 0.0074 - val_accuracy: 0.9965
Epoch 3/10
188/188 [==============================] - 12s 62ms/step - loss: 0.0057 - accuracy: 0.9985 - val_loss: 0.0081 - val_accuracy: 0.9975
Epoch 4/10
188/188 [==============================] - 12s 62ms/step - loss: 0.0101 - accuracy: 0.9972 - val_loss: 0.0046 - val_accuracy: 0.9990
Epoch 5/10
188/188 [==============================] - 13s 67ms/step - loss: 0.0017 - accuracy: 0.9997 - val_loss: 0.0033 - val_accuracy: 0.9990
Epoch 6/10
188/188 [==============================] - 12s 62ms/step - loss: 4.6956e-04 - accuracy: 1.0000 - val_loss: 0.0023 - val_accuracy: 0.9995
Epoch 7/10
188/188 [==============================] - 12s 65ms/step - loss: 0.0074 - accuracy: 0.9983 - val_loss: 0.0307 - val_accur

2nd Architeture

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[early_stop]
)


Found 6000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/20
188/188 [==============================] - 83s 417ms/step - loss: 0.2218 - accuracy: 0.9333 - val_loss: 1.2511 - val_accuracy: 0.5035
Epoch 2/20
188/188 [==============================] - 77s 411ms/step - loss: 0.0252 - accuracy: 0.9922 - val_loss: 6.8314 - val_accuracy: 0.5000
Epoch 3/20
188/188 [==============================] - 76s 406ms/step - loss: 0.0170 - accuracy: 0.9953 - val_loss: 0.8090 - val_accuracy: 0.5310
Epoch 4/20
188/188 [==============================] - 75s 400ms/step - loss: 0.0245 - accuracy: 0.9925 - val_loss: 0.0013 - val_accuracy: 0.9995
Epoch 5/20
188/188 [==============================] - 75s 397ms/step - loss: 0.0109 - accuracy: 0.9970 - val_loss: 0.0075 - val_accuracy: 0.9995
Epoch 6/20
188/188 [==============================] - 75s 399ms/step - loss: 0.0142 - accuracy: 0.9955 - val_loss: 0.0033 - val_accuracy: 0.9990
Epoch 7/20
188/188 [==========================